# Gerekliliklerin Kurulumları


In [7]:
%%capture --no-stderr
%pip install --quiet langchain-anthropic langchain-community langchain-core langgraph langchain-openai duckduckgo-search numexpr pydantic
%pip install -r setup-requirements.txt

# Ortam değişkenlerinin ayarlanması


In [8]:
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = ("X")

# Single Agent Oluşturma


In [9]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Agent oluşturma
memory = MemorySaver()
model = ChatOpenAI(model_name="gpt-4o-mini")
search = DuckDuckGoSearchResults(num_results=3)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Kod örneği - legacy kod refactoring örneği
code_to_review = """
# Eski stil hesaplama fonksiyonu
def calculate_metrics(data):
    total = 0
    count = 0
    errors = []
    
    for item in data:
        try:
            value = float(item['value'])
            total += value
            count += 1
        except (KeyError, ValueError) as e:
            errors.append(str(e))
    
    if count == 0:
        return {
            'average': 0,
            'total': 0,
            'error_count': len(errors),
            'errors': errors
        }
    
    return {
        'average': total / count,
        'total': total,
        'error_count': len(errors),
        'errors': errors
    }
"""

review_prompt = f"""Sen deneyimli bir Python geliştiricisisin. Bu kodu incele ve:
1. Kodun güçlü ve zayıf yönlerini tespit et
2. Modern Python özelliklerini kullanarak nasıl iyileştirilebileceğini açıkla
3. Performance analizi yap
4. Clean Code prensipleri açısından değerlendir
5. Test edilebilirlik açısından öneriler sun

İşte kod:
{code_to_review}

Her zaman Türkçe cevap ver.
"""

config = {"configurable": {"thread_id": "code_review_123"}}
print("### Kod İnceleme Analizi ###")
print("-" * 50)
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content=review_prompt)]}, config
):
    print(chunk)
    print("-" * 50)

print("#######################################")

if "agent" in chunk and "messages" in chunk["agent"]:
    for message in chunk["agent"]["messages"]:
        if hasattr(message, "content"):
            print(message.content)
    print("-" * 50)

improvement_prompt = """Yukarıdaki kod incelemesine ve Python'daki güncel best practice'lere 
dayanarak, bu kodun daha modern bir versiyonunu önerir misin? Özellikle:
1. Modern Python özellikleri (dataclasses, exception handling, type hints) kullanarak
2. SOLID prensiplerine uygun şekilde
3. Daha iyi test edilebilirlik için
4. Error handling ve logging mekanizmalarını güçlendirerek
5. Daha iyi dokümantasyon ve okunabilirlik için

Ayrıca modern Python kod yapısı için topluluk tarafından önerilen kaynakları da araştırıp paylaşır mısın?

Her zaman Türkçe cevap ver.
"""

print("\n### İyileştirilmiş Kod Önerisi ve Kaynaklar ###")
print("-" * 50)

# All Runnable objects implement a sync method called stream and an async variant called astream
# sync stream and async astream: a default implementation of streaming that streams the final output from the chain
# async astream_events and async astream_log these provide a way to stream both intermediate steps and final output from the chain
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content=improvement_prompt)]}, config
):
    print(chunk)
    print("-" * 50)

print("#######################################")

if "agent" in chunk and "messages" in chunk["agent"]:
    for message in chunk["agent"]["messages"]:
        if hasattr(message, "content"):
            print(message.content)
    print("-" * 50)

### Kod İnceleme Analizi ###
--------------------------------------------------
{'agent': {'messages': [AIMessage(content="Kodunuzu inceledim. Aşağıda güçlü ve zayıf yönleri, iyileştirme önerileri, performans analizi, Clean Code prensipleri açısından değerlendirme ve test edilebilirlik önerilerini bulabilirsiniz.\n\n### 1. Kodun Güçlü ve Zayıf Yönleri\n\n**Güçlü Yönler:**\n- Temel işlevselliği yerine getiriyor: Verileri alarak toplam ve ortalamayı hesaplayabiliyor.\n- Hataları yakalayabiliyor ve listeliyor.\n\n**Zayıf Yönler:**\n- Kod tekrarları (örneğin, hataların kaydedilmesi).\n- Hataları yakalamak için iki ayrı durum (KeyError ve ValueError) kullanılmış, ancak bu genelde daha yalın bir şekilde ele alınabilir.\n- Hata mesajları kullanıcıya sunulmadığı için, hata ayıklama veya kullanıcı deneyimi açısından yetersiz kalıyor.\n\n### 2. Modern Python Özellikleri ile İyileştirme\n\n- **Type Hinting:** Fonksiyonun giriş ve çıkış türlerini belirterek okunabilirliği artırabiliriz.\n- **List 

# Agent + Search Stream


In [10]:
news_agent = create_react_agent(model, tools, checkpointer=memory)

analysis_prompt = """Sen deneyimli bir araştırmacı ve analistsin. 

Türkiye'deki elektrikli araç pazarı hakkında detaylı bir analiz yapmanı istiyorum. Lütfen:

1. Son 6 aydaki pazar büyüklüğünü ve büyüme oranlarını araştır
2. Türkiye'de mevcut elektrikli araç modellerini ve fiyat aralıklarını belirle
3. Şarj istasyonu altyapısının mevcut durumunu değerlendir
4. Devlet teşvikleri ve düzenlemelerini incele
5. Pazarın önündeki fırsatları ve zorlukları analiz et

Bu bilgileri güncel kaynaklardan araştırarak kapsamlı bir rapor hazırla.

Her zaman Türkçe cevap ver.
"""

config = {"configurable": {"thread_id": "ev_market_analysis"}}
print("### Elektrikli Araç Pazar Analizi ###")
print("-" * 50)
for chunk in news_agent.stream(
    {"messages": [HumanMessage(content=analysis_prompt)]}, config
):
    print(chunk)
    print("-" * 50)

print("#######################################")

if "agent" in chunk and "messages" in chunk["agent"]:
    for message in chunk["agent"]["messages"]:
        if hasattr(message, "content"):
            print(message.content)
    print("-" * 50)

# Detaylı sorular
followup_prompt = """Teşekkürler. Şimdi lütfen şu konularda daha detaylı bilgi ver:

1. En çok satan 3 elektrikli araç modeli ve satış rakamları nedir?
2. Şarj istasyonlarının şehirlere göre dağılımı nasıl?
3. 2024 yılı için açıklanan yeni teşvik ve düzenlemeler var mı?
4. Yerli elektrikli araç üretimi konusundaki gelişmeler neler?

Lütfen en güncel verileri kullanarak yanıtla.

Her zaman Türkçe cevap ver.
"""

print("\n ### Detaylı Analiz ve Güncel Veriler ###")
print("-" * 50)
for chunk in news_agent.stream(
    {"messages": [HumanMessage(content=followup_prompt)]}, config
):
    print(chunk)
    print("-" * 50)

print("#######################################")

if "agent" in chunk and "messages" in chunk["agent"]:
    for message in chunk["agent"]["messages"]:
        if hasattr(message, "content"):
            print(message.content)
    print("-" * 50)

### Elektrikli Araç Pazar Analizi ###
--------------------------------------------------
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_s4K4W1u1rzJory8Ww5Rw6oKV', 'function': {'arguments': '{"query": "Türkiye elektrikli araç pazarı büyüklüğü 2023 son 6 ay"}', 'name': 'duckduckgo_results_json'}, 'type': 'function'}, {'id': 'call_GwDL4cJTpXtmnCgg3ftMMkEh', 'function': {'arguments': '{"query": "Türkiye elektrikli araç modelleri fiyat aralıkları 2023"}', 'name': 'duckduckgo_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 223, 'total_tokens': 303, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'log

# Hisse Sembol Taraması


In [11]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_community.chat_models import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun, tool


@tool
def search_ticker(company_name: str) -> str:
    """
    DuckDuckGo kullanarak belirli bir şirketin hisse senedi sembolünü ve güncel fiyatını arar.
    Bulunduğunda hem hisse senedi sembolünü hem de güncel hisse senedi fiyatını döndürür.
    """
    search = DuckDuckGoSearchRun()
    # İlk olarak ticker sembolünü arayalım
    ticker_results = search.run(f"{company_name} stock ticker symbol MarketWatch")

    # Sonra güncel fiyat için arama yapalım
    price_results = search.run(f"{company_name} current stock price real time")

    latest_news = search.run(f"{company_name} latest news")

    return f"""
Arama Sonuçları:

Ticker Bilgisi:
{ticker_results}

Fiyat Bilgisi:
{price_results}

Son Haberler:
{latest_news}

Her zaman Türkçe cevap ver.
"""


# Model ve araçları tanımlama
llm = ChatOpenAI(temperature=0.0, model_name="gpt-4o-mini")
tools = [search_ticker]

# Agent oluşturma
agent = initialize_agent(
    tools=tools, llm=llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)
# Agent'a komut verme
output = agent.run(
    """
    Aşağıdaki hissenin sembolünü, güncel fiyatını ve son haberlerini bul:

    Tesla
    
    Her zaman Türkçe cevap ver.
    """
)

print(output)

/var/folders/_x/1jnh7t8s7_90c39yscwczr700000gn/T/ipykernel_12605/1137177428.py:38: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.0, model_name="gpt-4o-mini")
/var/folders/_x/1jnh7t8s7_90c39yscwczr700000gn/T/ipykernel_12605/1137177428.py:42: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as



> Entering new AgentExecutor chain...
Tesla'nın hisse senedi sembolünü ve güncel fiyatını bulmak için arama yapmalıyım. Ayrıca son haberleri de kontrol etmem gerekecek. Öncelikle hisse senedi bilgilerini bulmalıyım.  
Action: search_ticker  
Action Input: Tesla  
Observation: 
Arama Sonuçları:

Ticker Bilgisi:
Detailed Quote for Saturday, January 18, 2025 07:48 AM; Company Name: Tesla Inc. Dow Jones Industry: Automobiles: Exchange: NAS: Shares Outstanding: 3,210,059,600 See the latest Tesla Inc stock price (NASDAQ:TSLA), ... MarketWatch Jan 22, ... Tesla's Trump bump fades as stock falls alongside rest of EV sector. Read: Tesla's market cap tops $1 trillion after another Trump-fueled jump for the stock. BofA analyst John Murphy reiterated his buy rating on the stock in a Thursday note to clients, raising his ... Get a real-time Tesla, Inc. (TSLA) stock price quote with breaking news, financials, statistics, charts and more. Get a real-time Tesla, Inc. (TSLA) stock price quote with br

In [13]:
from langchain.agents import AgentType, initialize_agent
from langchain.chains import LLMMathChain
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

primes = {998: 7901, 999: 7907, 1000: 7919}


class CalculatorInput(BaseModel):
    question: str = Field()


class PrimeInput(BaseModel):
    n: int = Field()


def is_prime(n: int) -> bool:
    if n <= 1 or (n % 2 == 0 and n > 2):
        return False
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0:
            return False
    return True


def get_prime(n: int, primes: dict = primes) -> str:
    return str(primes.get(int(n)))


async def aget_prime(n: int, primes: dict = primes) -> str:
    return str(primes.get(int(n)))


tools = [
    Tool(
        name="GetPrime",
        func=get_prime,
        description="`n`inci asal sayıyı döndürür",
        args_schema=PrimeInput,
        coroutine=aget_prime,
    ),
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Matematiksel ifadeleri hesaplamak için kullanılır",
        args_schema=CalculatorInput,
        coroutine=llm_math_chain.arun,
    ),
]

from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")

from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

question = "998., 999. ve 1000. asal sayıların çarpımı nedir?"

for step in agent_executor.iter({"input": question}):
    if output := step.get("intermediate_step"):
        action, value = output[0]
        if action.tool == "GetPrime":
            print(f"Kontrol ediyoruz {value} asal mı...")
            assert is_prime(int(value))
        _continue = input("Devam etmek istiyor musun (Y/n)?:\n") or "Y"
        if _continue.lower() != "y":
            break

/Users/dtunai/Desktop/Presentation/.venv/lib/python3.13/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new None chain...

Invoking: `GetPrime` with `{'n': 998}`


7901Kontrol ediyoruz 7901 asal mı...

Invoking: `GetPrime` with `{'n': 999}`


7907Kontrol ediyoruz 7907 asal mı...

Invoking: `GetPrime` with `{'n': 1000}`


7919Kontrol ediyoruz 7919 asal mı...

Invoking: `Calculator` with `{'question': '7901 * 7907 * 7919'}`




> Entering new LLMMathChain chain...
7901 * 7907 * 7919```text
7901 * 7907 * 7919
```
...numexpr.evaluate("7901 * 7907 * 7919")...

Answer: 494725326233
> Finished chain.
Answer: 494725326233998., 999. ve 1000. asal sayıların çarpımı 494725326233'tür.

> Finished chain.
